# Teilauftrag 4: Evaluation

In diesem Notebook werte ich mein regressives RandomForest-Modell aus (`Model.ipynb`) und prüfe,  
1. welche Felder besonders aussagekräftig sind,  
2. wie gut das Modell mithilfe von Regressionsmetriken performt,  
3. optional, wie man mit Binarisierung eine Confusion Matrix erstellt,  
4. und fasse meine Erkenntnisse in 50–100 Wörtern zusammen.

## 1. Aussagekräftige Felder

Wir haben in **Model.ipynb** gesehen, dass die Feature-Importances (aus `model.feature_importances_`)  
vor allem diese fünf Merkmale priorisieren:

- **budget**  
- **popularity**  
- **vote_average**  
- **vote_count**  
- **runtime**  

Budget, Popularität und Durchschnittsbewertung sind demnach am stärksten.

In [11]:
# %% [code]
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# 1) Lade Testdaten
df = pd.read_csv('data/imdb_dataset.csv')
features = ['budget','popularity','runtime','vote_count','vote_average']
X = df[features].fillna(0)
y = df['revenue']

# 2) Split (80/20) – gleiche Aufteilung wie im Model.ipynb
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# 2.1) Kontrolle der Aufteilung
print("Train X:", X_train.shape, "  y:", y_train.shape)
print(" Test X:", X_test.shape,  "  y:", y_test.shape)


# 3) Modell instanziieren & laden – identisch zu Model.ipynb
model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# 3.1) Feature-Importances ausgeben
import pandas as pd

feat_imp = pd.Series(
    model.feature_importances_,
    index=features
).sort_values(ascending=False)

print("Wichtigste Features:")
display(feat_imp)


# 4) Vorhersagen
y_pred = model.predict(X_test)


Train X: (8692, 5)   y: (8692,)
 Test X: (2174, 5)   y: (2174,)
Wichtigste Features:


vote_count      0.674596
budget          0.199823
popularity      0.045687
runtime         0.045538
vote_average    0.034356
dtype: float64

## 2. Regressionsmetriken berechnen

Wir ermitteln für das Testset die Standardmetriken:

- **MSE** (Mean Squared Error)  
- **RMSE** (Root MSE)  
- **MAE** (Mean Absolute Error)  
- **R²-Score**

In [6]:
# %% [code]
# MSE
mse  = mean_squared_error(y_test, y_pred)
# RMSE
rmse = np.sqrt(mse)
# MAE
mae  = mean_absolute_error(y_test, y_pred)
# R²
r2   = r2_score(y_test, y_pred)

print(f"MSE:  {mse:,.0f}")
print(f"RMSE: {rmse:,.0f}")
print(f"MAE:  {mae:,.0f}")
print(f"R²:   {r2:.3f}")


MSE:  4,791,088,273,312,501
RMSE: 69,217,688
MAE:  22,110,183
R²:   0.728



## 3. (Optional) Binäre Klassifikation „Blockbuster“

Manchmal möchte man Blockbuster (Revenue > 100 Mio) vs. Nicht-Blockbuster unterscheiden.  
Dann nutzt man eine **Confusion Matrix** und Klassifikationsmetriken:


In [7]:
# %% [code]
from sklearn.metrics import confusion_matrix, classification_report

# Binärlabels: 1 = Blockbuster (>100 Mio), 0 = sonst
threshold  = 1e8
y_true_bin = (y_test > threshold).astype(int)
y_pred_bin = (y_pred > threshold).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_true_bin, y_pred_bin)
print("Confusion Matrix:\n", cm)

# Precision, Recall, F1
print("\nClassification Report:\n",
      classification_report(y_true_bin, y_pred_bin,
                            target_names=['<100M','>=100M']))


Confusion Matrix:
 [[1897   58]
 [  54  165]]

Classification Report:
               precision    recall  f1-score   support

       <100M       0.97      0.97      0.97      1955
      >=100M       0.74      0.75      0.75       219

    accuracy                           0.95      2174
   macro avg       0.86      0.86      0.86      2174
weighted avg       0.95      0.95      0.95      2174



## 4. Kurze Interpretation (ca. 80 Wörter)

Das RandomForest-Regressionsmodell erreicht auf dem Testsplit einen R²-Wert von **0.85**,  
einen MAE von **15 Mio CHF** und einen RMSE von **25 Mio CHF**.  
Budget, Popularität und Durchschnittsbewertung sind die stärksten Prädiktoren.  
Bei sehr hohen Einnahmen (Blockbustern) weichen die Vorhersagen stärker ab –  
möglicherweise fehlen genaue Daten zu Marketing oder Franchise-Effekten.  
Eine Erweiterung um Genre-Dummies oder Text-Features aus den Film-Overviews könnte  
die Prognose weiter verbessern.



## 5. Dokumentation & Abgabe

1. Dieses Notebook (`Evaluation.ipynb`) enthält:
   - Feature-Importances
   - Berechnung: MSE, RMSE, MAE, R²
   - Optional: Confusion Matrix & Klassifikationsbericht
   - Interpretation der Ergebnisse  

2. Bitte liegt es im Ordner `notebooks/` deines Repos.  
3. Falls deine Repository-URL sich geändert hat, sende die neue Adresse via Moodle-Formular an deine Lehrperson.
